In [14]:
import psycopg2 # pip install psycopg2
import plotly.graph_objects as go # pip install plotly
import plotly.io as pio # save as image pip install kaleido
import pandas as pd

In [15]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="password",
)

In [16]:
def human_readable_format(number):
    number = float(number)
    if number >= 1e12:
        return f'{number / 1e12:.2f}T'  # Trillion
    elif number >= 1e9:
        return f'{number / 1e9:.2f}B'  # Billion
    elif number >= 1e6:
        return f'{number / 1e6:.2f}M'  # Million
    elif number >= 1e3:
        return f'{number / 1e3:.2f}K'  # Thousand
    else:
        return str(number)  # Less than thousand, return as is

In [17]:
cursor = conn.cursor()
query = """
SELECT 
    info->>'symbol' AS symbol,
    info->>'shortName' AS short_name,
    array_agg(
        json_build_object(
            'close_date', to_timestamp((key)::numeric / 1000), 
            'price', value::numeric
        )
    ) AS close_history
FROM stocks.yfinance y,
    json_each_text(five_year->'Close') AS close_data(key, value)
WHERE info->>'symbol' = 'AAPL'
GROUP BY symbol, short_name;
"""
cursor.execute(query)
result = cursor.fetchone()
cursor.close()

In [18]:
close_history = result[2]
dates = []
prices = []

for entry in close_history:
    date = entry['close_date']
    price = entry['price']
    
    dates.append(date)
    prices.append(price)

# Plotting with Plotly
fig = go.Figure(data=go.Scatter(x=dates, y=prices, mode='lines+markers'))

# Update layout for better readability
fig.update_layout(
    title="Stock Close Price Over Time (AAPL)",
    xaxis_title="Date",
    yaxis_title="Close Price (USD)",
    xaxis=dict(tickformat="%Y-%m-%d", tickangle=45),
    template="plotly_dark"
)

# Show the plot
fig.show()

In [19]:
fig.write_html("interactive_plot.html", include_plotlyjs='cdn', full_html=False)
fig.to_html(include_plotlyjs='cdn', full_html=False)
#fig.write_image("stock_close_price.png")

'<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.35.2.min.js"></script>                <div id="75b4bfee-10ff-4a6f-81cc-9e3d74ee42dd" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("75b4bfee-10ff-4a6f-81cc-9e3d74ee42dd")) {                    Plotly.newPlot(                        "75b4bfee-10ff-4a6f-81cc-9e3d74ee42dd",                        [{"mode":"lines+markers","x":["2020-01-01T00:00:00-05:00","2020-01-08T00:00:00-05:00","2020-01-15T00:00:00-05:00","2020-01-22T00:00:00-05:00","2020-01-29T00:00:00-05:00","2020-02-05T00:00:00-05:00","2020-02-12T00:00:00-05:00","2020-02-19T00:00:00-05:00","2020-02-26T00:00:00-05:00","2020-03-04T00:00:00-05:00","2020-0

In [20]:
cursor = conn.cursor()

query = """
SELECT 
    info->>'symbol' AS symbol,
    info->>'shortName' AS short_name,
    info->>'sector' AS sector,
    (info->>'marketCap')::numeric AS market_cap
FROM stocks.yfinance y;
"""
cursor.execute(query)

data = cursor.fetchall()
cursor.close()

In [21]:
cursor = conn.cursor()

df = pd.DataFrame(data, columns=['symbol', 'short_name', 'sector', 'market_cap'])

# Group by sector and sum the market cap
sector_market_cap = df.groupby('sector')['market_cap'].sum().reset_index()

# Create a Plotly pie chart
fig = go.Figure(go.Pie(
    labels=sector_market_cap['sector'],
    values=sector_market_cap['market_cap'],
    hole=0.3,  # Optional: Adds a hole in the middle for a donut chart effect
))

# Customize the layout
fig.update_layout(
    title="Market Cap by Sector",
    template="plotly_dark",  # Optional: dark theme for the chart
)

cursor.close()

# Show the pie chart
fig.show()

In [22]:
cursor = conn.cursor()
query = """
SELECT 
    info->>'symbol' AS symbol,
    info->>'shortName' AS short_name,
    info->>'sector' AS sector,
    (info->>'marketCap')::numeric AS market_cap
FROM stocks.yfinance y
WHERE info->>'sector' = 'Technology'  -- Filter for Technology sector
ORDER BY market_cap DESC  -- Order by market cap in descending order
LIMIT 20;  -- Limit to top 20 companies
"""

cursor.execute(query)

data = cursor.fetchall()

In [23]:
# Create a pandas DataFrame from the query results
df = pd.DataFrame(data, columns=['symbol', 'short_name', 'sector', 'market_cap'])

# Filter out some specific companies (optional)
df = df[~df['symbol'].isin(['AAPL', 'MSFT', 'NVDA'])]

# Create a Plotly pie chart for the top 20 Technology sector companies
fig = go.Figure(go.Pie(
    labels=df['short_name'],  # Use the short name of the company for labels
    values=df['market_cap'],  # Market cap as the value for each slice
    hole=0.3,  # Optional: Adds a hole in the middle for a donut chart effect
    textinfo='none',  # Show only label and value (no percentage on the right)
    hoverinfo='label+value+percent',  # Show label, value, and percentage on hover
))

# Customize the layout
fig.update_layout(
    title="Top 20 Companies by Market Cap in the Technology Sector",
    template="plotly_dark",  # Optional: dark theme for the chart
)

# Show the pie chart
fig.show()

In [24]:
cursor = conn.cursor()

query = """
SELECT 
    info->>'sector' AS sector,
    SUM((info->>'marketCap')::numeric) AS total_market_cap
FROM stocks.yfinance y
WHERE info->>'sector' IS NOT NULL  -- Exclude rows where sector is null
GROUP BY info->>'sector'
ORDER BY total_market_cap DESC;  -- Sort by market cap
"""

# Execute the query
cursor.execute(query)

# Fetch the results
data = cursor.fetchall()

# Close the cursor and connection
cursor.close()

In [25]:
# Create a pandas DataFrame from the query results
df = pd.DataFrame(data, columns=['sector', 'total_market_cap'])

df['formatted_market_cap'] = df['total_market_cap'].apply(human_readable_format)

# Create a Plotly pie chart for market cap by sector
fig = go.Figure(go.Pie(
    labels=df['sector'],  # Use the sector names for labels
    values=df['total_market_cap'],  # Market cap as the value for each slice
    hole=0.3,  # Optional: Adds a hole in the middle for a donut chart effect
    textinfo='none',  # Do not show any text inside the chart
    hoverinfo='label+text+percent',  # Show label, value, and percentage on hover
    text=df['formatted_market_cap'],
))

# Customize the layout
fig.update_layout(
    title="Market Cap by Sector",
    template="plotly_dark",
)

# Show the pie chart
fig.show()